In [1]:
# pipenv install pandas plotly scikit-learn optuna ipywidgets ipykernel nbformat matplotlib pingouin gradio

# EDA e Visualização de Dados
import pandas as pd
import plotly.express as px
pd.set_option('display.float_format', lambda x: '%.2f' % x)


# ML
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, pairwise_distances
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer

# Otimização de HP
import optuna

### Carregar os Dados

In [2]:
# Carga de Dados
df_clients = pd.read_csv('./datasets/clients_pj.csv')

In [3]:
df_clients.head(10)

,atividade_economica,faturamento_mensal,numero_de_funcionarios,localizacao,idade,inovacao
0,Comércio,713109.95,12,Rio de Janeiro,6,1
1,Comércio,790714.38,9,São Paulo,15,0
2,Comércio,1197239.33,17,São Paulo,4,9
3,Indústria,449185.78,15,São Paulo,6,0
4,Agronegócio,1006373.16,15,São Paulo,15,8
5,Serviços,1629562.41,16,Rio de Janeiro,11,4
6,Serviços,771179.95,13,Vitória,0,1
7,Serviços,707837.61,16,São Paulo,10,6
8,Comércio,888983.66,17,Belo Horizonte,10,1
9,Indústria,1098512.64,13,Rio de Janeiro,9,3


### EDA

In [4]:
# distribuição da variável inovação
percentage_inovation = df_clients.value_counts('inovacao')/ len(df_clients)  * 100
px.bar(percentage_inovation, color=percentage_inovation.index)

In [5]:
# Testar ANOVA (Análise de Variância)
# Veriricar se há variações significativas na média de faturamento mensal para diferentes níveis de inovação
# Suposições / Pressupostos:
# - Observações indepednetes
# - Variável dependente é continua
# - Segue uma distribuição normal
# - Homogeneidade das variâncias
# - Amostrar sejam de tamanhos iguais

In [6]:
# Checar se a s variâncias (faturamento) entre os grupos (inovação) são homogêneas
# Aplicar Testes de Bartlett
# H0 - Variâncias são iguais
# H1 - Variâncias não são iguais

from scipy.stats import bartlett

# Separando os dados de faturamento em grupos com base na coluna 'inovacao'

data_group = [df_clients['faturamento_mensal'][df_clients['inovacao'] == group] for group in df_clients['inovacao'].unique()]

# Executar o teste de Bartlett
bartlett_test_statistic, bartlett_p_value = bartlett(*data_group)

# Exibindo os resultados
print(f'Estátistica do Teste de Bartlett:  {bartlett_test_statistic}')
print(f'P-Value do Teste de Bartlett:  {bartlett_p_value}')

Estátistica do Teste de Bartlett:  10.901203117231173
P-Value do Teste de Bartlett:  0.28254182954905804


In [7]:
# Executar o teste de Shapiro-Wilk
# Verificar se os dados seguem uma distribuição normal
# H0 - Segue uma distribuição normal
# H1 - Não segue uma distribuição normal

from scipy.stats import shapiro

# Executar o teste de Bartlett
shapiro_test_statistic, shapiro_p_value = shapiro(df_clients['faturamento_mensal'])

# Exibindo os resultados
print(f'Estátistica do Teste de Shapiro Wilk:  {shapiro_test_statistic}')
print(f'P-Value do Teste de Shapiro Wilk:  {shapiro_p_value}')

Estátistica do Teste de Shapiro Wilk:  0.9959857602472711
P-Value do Teste de Shapiro Wilk:  0.23513451034389005


In [8]:
# Aplicar a ANOVA de Welch, pois as amostras são de tamanhos diferentes
# H0 - Não há diferenças significativas entre as médias dos grupos
# H1 - Há pelo menos uma diferença significativa entre as médias dos grupos
from pingouin import welch_anova

aov = welch_anova(dv='faturamento_mensal', between='inovacao', data=df_clients)


# Exibindo os resultados
print(f"Estátistica do Teste de ANOVA Welch: {aov.loc[0, 'F']}")
print(f"P-Value do Teste de ANOVA Welch: {aov.loc[0, 'p-unc']}")

Estátistica do Teste de ANOVA Welch: 1.1269836194061693
P-Value do Teste de ANOVA Welch: 0.34526211273911467


### Treinar o algoritmo K-Means

In [9]:
# Selecionar as colunas para clusterização
X = df_clients.copy()

# Separando variáveis numéricas, categoricas e ordinais
numeric_features = ['faturamento_mensal', 'numero_de_funcionarios', 'idade']
categorical_features = ['localizacao', 'atividade_economica']
ordinal_features = ['inovacao']

# Aplicar Transformações por tipo
numeric_transformer = StandardScaler()
categorical_transformer= OneHotEncoder()
ordinal_transformer = OrdinalEncoder()


preprocessor = ColumnTransformer(
  transformers=[
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, categorical_features),
    ('ord', ordinal_transformer, ordinal_features)
  ]
)


# Transformar os dados
X_transformed = preprocessor.fit_transform(X)

In [10]:
X_transformed

array([[-0.74634498, -0.54179191, -1.10058849, ...,  0.        ,
         0.        ,  1.        ],
       [-0.56165548, -1.5035527 ,  1.94344851, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.40582654,  1.06114274, -1.77704115, ...,  0.        ,
         0.        ,  9.        ],
       ...,
       [ 2.8196246 , -1.18296577,  0.25231684, ...,  0.        ,
         1.        ,  0.        ],
       [ 1.03321411, -0.54179191, -1.43881482, ...,  0.        ,
         0.        ,  3.        ],
       [-2.03011486, -0.22120498, -1.77704115, ...,  1.        ,
         0.        ,  9.        ]])

In [11]:
# Optuna para Otimização de Hiperparâmetros
def kmeans_objective(trial):
  # Definir os hiperparâmetros a serem ajustados
  n_clusters = trial.suggest_int('n_clusters', 3, 10)
  distance_metric = trial.suggest_categorical('distance_metric', ['euclidean', 'minkowski'])

  # Criar o Modelo
  model_kmeans = KMeans(n_clusters=n_clusters, random_state=51)

  # Treinar o modelo
  model_kmeans.fit(X_transformed)

  # Calculador o Silhouette Score
  distances = pairwise_distances(X_transformed, metric=distance_metric)
  silhouette_avg = silhouette_score(distances, model_kmeans.labels_)

  return silhouette_avg

In [12]:
# Criar um estudo do Optuna
search_space = {'n_clusters': [3,4,5,6,7,8,9,10], 'distance_metric': ['euclidean', 'minkowski']}

sampler = optuna.samplers.GridSampler(search_space=search_space)
study_kmeans = optuna.create_study(direction='maximize', sampler=sampler)

# Rodar o estudo
study_kmeans.optimize(kmeans_objective, n_trials=100)

[I 2025-03-26 01:09:19,173] A new study created in memory with name: no-name-75f31570-7b24-4e33-a1e2-8f7ef4b91cd9
c:\Users\felipe.fardo\.virtualenvs\k_means-ZRn9F5lI\lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning:

Could not find the number of physical cores for the following reason:
[WinError 2] O sistema não pode encontrar o arquivo especificado
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.

  File "c:\Users\felipe.fardo\.virtualenvs\k_means-ZRn9F5lI\lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
  File "C:\Users\felipe.fardo\.pyenv\pyenv-win\versions\3.9.6\lib\subprocess.py", line 505, in run
    with Popen(*popenargs, **kwargs) as process:
  File "C:\Users\felipe.fardo\.pyenv\pyenv-win\versions\3.9.6\lib\subprocess.py", line 951, in __init__
    self._execute_child(args, execu

In [13]:
# Melhor configuração encontrada pelo Optuna
best_params = study_kmeans.best_params

# Instanciando o modelo K-Means com melhores parâmetros

best_kmeans = KMeans(n_clusters=best_params['n_clusters'], random_state=51)
best_kmeans.fit(X_transformed)

# Calculador o Silhouette Score
distances = pairwise_distances(X_transformed, metric=best_params['distance_metric'])
best_silhouette = silhouette_score(distances, best_kmeans.labels_)

print(f"k (Número de Clusters): {best_params['n_clusters']}")
print(f"Métrica de Distância Selecionada: {best_params['distance_metric']}")
print(f"Silhouette Score: {best_silhouette}")

k (Número de Clusters): 3
Métrica de Distância Selecionada: euclidean
Silhouette Score: 0.4445458290999088


In [14]:
# Criar coluna com cluster escolhido
df_clients['cluster'] = best_kmeans.labels_

In [15]:
# Visualizar os primeiros registros
df_clients.head(10)

,atividade_economica,faturamento_mensal,numero_de_funcionarios,localizacao,idade,inovacao,cluster
0,Comércio,713109.95,12,Rio de Janeiro,6,1,0
1,Comércio,790714.38,9,São Paulo,15,0,0
2,Comércio,1197239.33,17,São Paulo,4,9,1
3,Indústria,449185.78,15,São Paulo,6,0,0
4,Agronegócio,1006373.16,15,São Paulo,15,8,1
5,Serviços,1629562.41,16,Rio de Janeiro,11,4,2
6,Serviços,771179.95,13,Vitória,0,1,0
7,Serviços,707837.61,16,São Paulo,10,6,1
8,Comércio,888983.66,17,Belo Horizonte,10,1,0
9,Indústria,1098512.64,13,Rio de Janeiro,9,3,2


### Visualizar Resultados

In [16]:
# Cruzar idade e faturamento, apresentando os clusters
px.scatter(df_clients, x='idade', y='faturamento_mensal', color='cluster')

In [17]:
# Cruzar inovacao e faturamento, apresentando os clusters
px.scatter(df_clients, x='inovacao', y='faturamento_mensal', color='cluster')

In [18]:
# Cruzar numero_de_funcionarios e faturamento, apresentando os clusters
px.scatter(df_clients, x='numero_de_funcionarios', y='faturamento_mensal', color='cluster')

### Salvar o Modelo e o Pipeline de Transformação

In [19]:
import joblib

# Salvar o modelo
joblib.dump(best_kmeans, 'model_clustering_clients.pkl')

# Salvar o Pipeline
joblib.dump(preprocessor, 'pipeline_clustering.pkl')

['pipeline_clustering.pkl']

### Aplicação Batch no Gradio

In [20]:
import gradio as gr

model = joblib.load('./model_clustering_clients.pkl')

preprocessor = joblib.load('./pipeline_clustering.pkl')

def clustering(archive):
  # Carregar o CSV em um Dataframe
  df_organizations = pd.read_csv(archive.name)

  # Transformar os dados do DF para o formato que o KMeans precisa
  X_transformed = preprocessor.fit_transform(df_organizations)

  # Treinar o modelo
  model.fit(X_transformed)

  # Criar a coluna de cluster no DF
  df_organizations['cluster'] = model.labels_
  df_organizations.to_csv('./clusters.csv', index=False)

  return './clusters.csv'


In [21]:
# Criar a interface
app = gr.Interface(
  clustering,
  gr.File(file_types=['.csv']),
  'file'
)

# Rodar a aplicação
app.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
